In [1]:
from collections import namedtuple
import csv
import math
import numpy as np
import matplotlib.pyplot as plt

#from module import multiple_reflection as mr

import sys
sys.path.append('..')

from module import solar_radiation_of_glazing as srg
from module import oblique_incidence_property as oip
from module import incident_solar_radiation_on_glazing as isrog

# 各層の吸収日射量を計算するモジュール

## 1. Functions  

開口面日射量（直達・天空・地表面反射）とグレージング複合体各層の日射吸収率から各層の吸収日射量を計算する。

### 入力値  

$ I_{T,b} $ ：斜面（開口面）直達日射量 (W/m<sup>2</sup>)  
$ I_{T,d} $ ：斜面（開口面）天空日射量 (W/m<sup>2</sup>)  
$ I_{T,r} $ ：斜面（開口面）地表面反射日射量 (W/m<sup>2</sup>)  
$ \alpha_{glz,b,j} $ ：直達日射に対する層$j$の日射吸収率  
$ \alpha_{glz,d,j,l,m} $ ：天空上の微小要素からの入射日射に対する層$j$の日射吸収率  
$ \alpha_{glz,r,j,l,m} $ ：地表面上の微小要素からの入射日射に対する層$j$の日射吸収率  
$ F_{d,l,m} $ ：天空日射量に対する天空上の微小要素からの天空日射量の割合  
$ F_{r,l,m} $ ：地表面反射日射量に対する地表面上の微小要素からの地表面反射日射量の割合  

### 出力値  

$ I_{\alpha,j} $ ：層$j$での吸収日射量 (W/m<sup>2</sup>)  

### 計算方法  

#### データ構造と値の範囲  

$ \alpha_{glz,b,j} $・$ \alpha_{glz,d,j} $・$ \alpha_{glz,r,j} $ の組み合わせをクラス『ata＿input』とする。  

#### 層$j$での吸収日射量の計算  

層$j$での吸収日射量の計算方法を以下に示す。

$$
\begin{eqnarray}
 &\displaystyle I_{\alpha,j} = I_{T,b} \cdot \alpha_{glz,b,j} + I_{T,d} \cdot \alpha_{glz,d} + I_{T,r} \cdot \alpha_{glz,r}& \qquad\qquad\text{(1)} \nonumber\\
 \nonumber \\
 &\displaystyle \alpha_{glz,d} = \sum_{l=0}^{N_{d,lat}-1}\sum_{m=0}^{N_{d,lon}-1} F_{d,l,m} \cdot \alpha_{glz,d,j,l,m}& \qquad\qquad\text{(2)} \nonumber\\
 \nonumber \\
 &\displaystyle \alpha_{glz,r} = \sum_{l=0}^{N_{r,lat}-1}\sum_{m=0}^{N_{r,lon}-1} F_{r,l,m} \cdot \alpha_{glz,r,j,l,m}& \qquad\qquad\text{(3)} \nonumber\\
\end{eqnarray}
$$

#### Example

In [2]:
if __name__ == '__main__':

    #面材の種類
    gl_type = [0, 0, 1]                #各層のグレージング種類（0：ガラス、1：ロールスクリーン、2：横型ブラインド、3：縦型ブラインド）
    
    #面材ごとの光学特性
    gl_in = [oip.GlassInput(0.815, 0.815, 0.072, 0.072, 0, False, False),
             oip.GlassInput(0.815, 0.815, 0.072, 0.072, 0, False, False),
             oip.RoleInput(0.3, 0.3, 0.63, 0.63)]

    #斜面日射量
    ita_in = isrog.ItaInput(
        itb=2.6840261248 * 10. ** 6. / 3600.,
        itd=0.1181932153 * 10. ** 6. / 3600.,
        itr=0.094 * 10. ** 6. / 3600.
    )

    #直達日射の入射角
    phi_b = 31.6403837639

    #天空分割数
    Nd_lat = 90
    Nd_lon = 180

    #地表面反射分割数
    Nr_lat = 90
    Nr_lon = 180

    abs_b = srg.actual_abs_beam(gl_in, phi_b)
    abs_d = srg.actual_abs_diffuse(gl_in, Nd_lat, Nd_lon)
    abs_r = srg.actual_abs_reflect(gl_in, Nr_lat, Nr_lon)
    
    I_abs = abs_b * ita_in.itb + abs_d * ita_in.itd + abs_d * ita_in.itr

    print (I_abs)

[139.94031065 125.26867442  39.32484644]
